## Web Scrapy of US City Metadata
This project is aiming in web scrapy by querying all city metadata from [uszip.com](https://www.uszip.com/). For each city, we want know their basci information, households, industry, occupation, etc., and output them to a csv file. 

There are two phases in this projects. The first one is use all state code (eg. CA) to get all zipcodes, and save this zip code file to a csv for further use. 

In the second phase, we will do a data processing. Since a zip code could be located to several cities, it's important to know which city is we are targeting given the zip code. Therefore, we use the city names from **zip-code-city-state-county_table.csv** and retrieve the default city name from it. In the end, we designed a spider to query all metadata of all cities and output the result with headers and each city as a row to a csv file.

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

## Fetch All ZIP Codes Under a State

In [2]:
state_codes = [
    'AL', 
    'AK', 
    'AZ', 
    'AR', 
    'CA', 
    'CO', 
    'CT', 
    'DE', 
    'DC',
    'FL', 
    'GA', 
    'HI', 
    'ID', 
    'IL', 
    'IN', 
    'IA', 
    'KS', 
    'KY', 
    'LA', 
    'ME', 
    'MD', 
    'MA', 
    'MI', 
    'MN', 
    'MS', 
    'MO', 
    'MT', 
    'NE', 
    'NV', 
    'NH', 
    'NJ', 
    'NM', 
    'NY', 
    'NC', 
    'ND', 
    'OH', 
    'OK', 
    'OR', 
    'PA', 
    'PR', 
    'RI', 
    'SC', 
    'SD', 
    'TN', 
    'TX', 
    'UT', 
    'VT', 
    'VA', 
    'VI', 
    'WA', 
    'WV',
    'WI',
    'WY'
]
us_zip_base_url = 'https://www.uszip.com'
us_zip_start_urls = [f'{us_zip_base_url}/state/{state_code}' for state_code in state_codes]

In [3]:
"""
{
    "<state_code>": {
        "<county_name>": [<zip_code>]
    }
}
"""
from collections import defaultdict
state_to_zips_dict = defaultdict(lambda: defaultdict(lambda: []))

In [4]:
import re
zip_code_format = r'^\d{5}$'

In [5]:
"""
Retrieve zip codes under a state and store them to state_to_zips_dict
"""
class USZipCodeSpider(scrapy.Spider):
    name = 'us_city_zipcode'
    start_urls = us_zip_start_urls
    
    def parse(self, response):
        for state in response.xpath('//*[@class="table tdata"]'):
            o.append(state)
            county_list = state.xpath('//tbody//tr')
            for county in county_list:
                county_name = county.xpath('td[2]//text()')[0].get().strip(',')
                state_code = county.xpath('td[2]//text()')[1].get().strip()
                
                zip_codes = [code.get() for code in county.xpath('td[3]//text()')]
                valid_zip_codes = []
                for zip_code in zip_codes:
                    if re.search(zip_code_format, zip_code):
                        valid_zip_codes.append(zip_code)

                state_to_zips_dict[state_code][county_name].extend(valid_zip_codes)

In [6]:
import os.path
from os import path

if not path.exists('state_to_zip.csv'):
    print('Start to parse zipcodes')
    fetch_zip_code_process = CrawlerProcess({'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'})
    fetch_zip_code_process.crawl(USZipCodeSpider)
    fetch_zip_code_process.start()
    
    # write results to csv file
    # format: <state>, <city>, <zip_code_list>
    # example: DE, Camden, 19901-19904-19934 

    import csv
    with open('state_to_zip.csv', 'w') as f:
        f.write("State,City,Zip_Code_List\n")
        for state_code, state_metadata in state_to_zips_dict.items():
            print(f'Writing info for state {state_code}')
            for city, zip_codes in state_metadata.items():
                zip_code_list = '-'.join(zip_codes)
                f.write("%s,%s,%s\n" % (state_code, city, zip_code_list))
else:
    print('File state_to_zip.csv already exist')

File state_to_zip.csv already exist


## Fetch County Metadata By Zip Code

In [7]:
# read all available zipcodes from uszip.com
import csv
zip_codes = set()
with open('state_to_zip.csv', 'r') as f:
    csv_reader = csv.reader(f)
    reader = next(csv_reader)
    for row in csv_reader:
        # split the zip code and remove empty zipcode before joining them to the list
        codes = list(filter(None, row[2].split('-')))
        zip_codes.update(codes)
zip_codes = list(zip_codes)

city_metadata_urls = [f'{us_zip_base_url}/zip/{zip_code}' for zip_code in zip_codes]

In [8]:
# read desired zipcodes, city name and state information
zip_code_to_city_state_county = {}
with open('zip-code-city-state-county_table.csv', 'r') as f:
    for line in csv.DictReader(f): 
        zip_code = line['zip']
        zip_code_to_city_state_county[zip_code] = dict(line)
len(zip_code_to_city_state_county)

33097

In [10]:
""" format: 
{
    zipcode_00115: {
        "city_metadata": {                        # this part is made by us
            "state_id": 'CA', city: 'Sunnyvale', 'county_name': 'Santa Clara'
        }, 
        basic info": {}, "Households": {}....  # these parts are from uszip.com
    }
}
"""
output = {}

In [11]:
# since scrapy cannot do xpath query to a portion of html
# we need to use other package for local search
from lxml import html
from collections import OrderedDict

class CityMetadataSpider(scrapy.Spider):
    name = 'us_city_metadata'
    start_urls = city_metadata_urls
    
    def __init__(self):
        self.available_section_titles = ['Basic info', 'Households',
                                'Housing units', 'Age', 'Employment status', 
                               'Occupation', 'Industry',
                               'Class of worker', 'Income']
    
    # parsing from the back to avoid any missing fields
    def parse_section(tree):
        res = []
        title_idx = len(tree.xpath('//dt')) - 1
        value_idx = len(tree.xpath('//dd')) - 1
        while value_idx >= 0:
            name = tree.xpath('//dt')[title_idx].text
            if not name: 
                name = tree.xpath(f'//dt[{title_idx + 1}]/strong/text()')[0]
            value = tree.xpath('//dd')[value_idx].text
            title_idx -= 1
            value_idx -= 1
            res.append((name, value))
        res.reverse()
        return res

    def parse(self, response):
        # fetch basic info, gender, households, etc.
        zip_code = response.xpath('//*[@id="content-body"]/div/div[2]/hgroup/h3/strong/text()').get()
        city = response.xpath('//*[@id="content-body"]/div/div[2]/hgroup/h2/strong/text()').get().strip(', ')
        state_id = response.xpath('//*[@id="content-body"]/div/div[2]/hgroup/h2/strong/a/text()').get()
        
        # use city and state_id from our side
        if zip_code in zip_code_to_city_state_county:
            city = zip_code_to_city_state_county[zip_code]['city']
            state_id = zip_code_to_city_state_county[zip_code]['state_id']
        city_metadata = [ ("zip_code", zip_code), ("city", city), ("state_id", state_id) ]
        
        section_infos = OrderedDict({ "city_metadata": city_metadata })
        
        for i, section in enumerate(response.xpath('//*[@id="content-body"]').xpath('//dl')): 
            # convert the section content to HTML string
            # the html package will parse it, and we can do futher analysis
            tree = html.fromstring(section.get())
            
            if i == 0:
                section_name = 'Basic info'
            else:
                section_name = tree.xpath('//h3/text()')[0].strip(":")
            if section_name in self.available_section_titles:
                section_info = CityMetadataSpider.parse_section(tree)
                section_infos[section_name] = section_info
        output[zip_code] = section_infos

In [12]:
fetch_metadata_process = CrawlerProcess({'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'})
fetch_metadata_process.crawl(CityMetadataSpider)
fetch_metadata_process.start()

2021-01-17 12:24:23 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-01-17 12:24:23 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 19.2.1, Python 3.7.5 (default, Nov  1 2019, 02:16:38) - [Clang 10.0.0 (clang-1000.11.45.5)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Darwin-19.3.0-x86_64-i386-64bit
2021-01-17 12:24:23 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-01-17 12:24:23 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2021-01-17 12:24:23 [scrapy.extensions.telnet] INFO: Telnet Password: 2cd6cd4c1c230b81
2021-01-17 12:24:23 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2021-01-17 12:24:23

2021-01-17 12:24:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/81067> (referer: None)
2021-01-17 12:24:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/76570> (referer: None)
2021-01-17 12:24:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/72167> (referer: None)
2021-01-17 12:24:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/63744> (referer: None)
2021-01-17 12:24:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/06902> (referer: None)
2021-01-17 12:24:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/10004> (referer: None)
2021-01-17 12:24:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/28906> (referer: None)
2021-01-17 12:24:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/97487> (referer: None)
2021-01-17 12:24:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https

2021-01-17 12:24:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/30678> (referer: None)
2021-01-17 12:24:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/13325> (referer: None)
2021-01-17 12:24:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/54889> (referer: None)
2021-01-17 12:24:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/75246> (referer: None)
2021-01-17 12:24:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/89815> (referer: None)
2021-01-17 12:24:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/46985> (referer: None)
2021-01-17 12:24:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/31305> (referer: None)
2021-01-17 12:24:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/21222> (referer: None)
2021-01-17 12:24:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https

2021-01-17 12:24:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/27051> (referer: None)
2021-01-17 12:24:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/60707> (referer: None)
2021-01-17 12:24:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/78737> (referer: None)
2021-01-17 12:24:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/81029> (referer: None)
2021-01-17 12:24:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/42102> (referer: None)
2021-01-17 12:24:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/08242> (referer: None)
2021-01-17 12:24:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/25846> (referer: None)
2021-01-17 12:24:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/47834> (referer: None)
2021-01-17 12:24:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https

2021-01-17 12:24:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/84775> (referer: None)
2021-01-17 12:24:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/34788> (referer: None)
2021-01-17 12:24:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/59079> (referer: None)
2021-01-17 12:24:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/19963> (referer: None)
2021-01-17 12:24:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/58448> (referer: None)
2021-01-17 12:24:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/48182> (referer: None)
2021-01-17 12:24:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/93933> (referer: None)
2021-01-17 12:24:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/66873> (referer: None)
2021-01-17 12:24:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https

...

2021-01-17 13:36:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/64507> (referer: None)
2021-01-17 13:36:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/98407> (referer: None)
2021-01-17 13:36:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/50118> (referer: None)
2021-01-17 13:36:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/11436> (referer: None)
2021-01-17 13:36:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/19533> (referer: None)
2021-01-17 13:36:23 [scrapy.extensions.logstats] INFO: Crawled 27241 pages (at 417 pages/min), scraped 0 items (at 0 items/min)
2021-01-17 13:36:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/68768> (referer: None)
2021-01-17 13:36:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/99149> (referer: None)
2021-01-17 13:36:23 [scrapy.core.engine] DEBUG: Crawled (200

2021-01-17 13:36:33 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/97426> (referer: None)
2021-01-17 13:36:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/46723> (referer: None)
2021-01-17 13:36:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/90095> (referer: None)
2021-01-17 13:36:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/93730> (referer: None)
2021-01-17 13:36:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/98326> (referer: None)
2021-01-17 13:36:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/25405> (referer: None)
2021-01-17 13:36:35 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/64653> (referer: None)
2021-01-17 13:36:35 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/58558> (referer: None)
2021-01-17 13:36:35 [scrapy.core.engine] DEBUG: Crawled (200) <GET https

2021-01-17 13:36:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/63034> (referer: None)
2021-01-17 13:36:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/29625> (referer: None)
2021-01-17 13:36:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/51552> (referer: None)
2021-01-17 13:36:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/48240> (referer: None)
2021-01-17 13:36:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/93458> (referer: None)
2021-01-17 13:36:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/38339> (referer: None)
2021-01-17 13:36:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/61449> (referer: None)
2021-01-17 13:36:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/26175> (referer: None)
2021-01-17 13:36:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET https

2021-01-17 13:36:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/26430> (referer: None)
2021-01-17 13:36:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/34698> (referer: None)
2021-01-17 13:36:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/37705> (referer: None)
2021-01-17 13:36:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/89052> (referer: None)
2021-01-17 13:36:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/23523> (referer: None)
2021-01-17 13:36:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/38230> (referer: None)
2021-01-17 13:36:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/49639> (referer: None)
2021-01-17 13:36:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/49042> (referer: None)
2021-01-17 13:36:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https

2021-01-17 13:37:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/85534> (referer: None)
2021-01-17 13:37:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/53574> (referer: None)
2021-01-17 13:37:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/87327> (referer: None)
2021-01-17 13:37:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/92058> (referer: None)
2021-01-17 13:37:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/56082> (referer: None)
2021-01-17 13:37:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/74467> (referer: None)
2021-01-17 13:37:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/56270> (referer: None)
2021-01-17 13:37:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.uszip.com/zip/57555> (referer: None)
2021-01-17 13:37:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https

## Write Result to CSV

In [13]:
# get headers from the first city
first_city_zip_code = list(output.keys())[0]

headers = []
city = output[first_city_zip_code]
for section in list(city.values()):
    for title, value in section:
        headers.append(title)
headers

['zip_code',
 'city',
 'state_id',
 'Total population',
 'Housing units',
 'Land area',
 'Density',
 'Water area',
 'Total households',
 'Family households',
 'Nonfamily households',
 'Average household size',
 'Average family size',
 'Total housing units',
 'Occupied housing units',
 'Owner-occupied',
 'Renter-occupied',
 'Vacant housing units',
 'Under 5 years',
 '5 to 9 years',
 '10 to 14 years',
 '15 to 19 years',
 '20 to 24 years',
 '25 to 34 years',
 '35 to 44 years',
 '45 to 54 years',
 '55 to 59 years',
 '60 to 64 years',
 '65 to 74 years',
 '75 to 84 years',
 '85 years and over',
 'Median age',
 'Population',
 'Civilian labor force',
 'Employed',
 'Unemployed',
 'Armed Forces',
 'Not in labor force',
 'Percent Unemployed',
 'Civilian employed population',
 'Management / business / science / arts',
 'Service',
 'Sales / office',
 'Natural resources / construction / maintenance',
 'Production / transportation / material moving',
 'Civilian employed population',
 'Agriculture / f

In [14]:
with open('city_metadata.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    
    for city in output.values():
        flatten_values = []
        for section in city.values():
            for _, value in section:
                flatten_values.append(value)
        writer.writerow(flatten_values)